In [33]:
import numpy as np
import tensorflow as tf
import keras 
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM

def gen_rnn_data(n):
    all_text = ""
    with open('../data/shakespeare.txt') as inp:
        for line in inp:
            all_text += line[:-1]
    seqs = []
    for i in range(40, len(all_text), n):
        seqs.append(all_text[i-40:i+1])
    return seqs  


def load_doc(filename):
    file = open(filename, 'r')s
    text = file.read()
    file.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

lines1 = gen_rnn_data(10)
raw_text1 = '\n'.join(lines1)

# Creating an encoding for each encountered character
chars1 = sorted(list(set(raw_text1)))
mapping = dict((c, i) for i, c in enumerate(chars1))
def train_model(seq_skip, n_nodes, n_epochs, model_filename):
    lines = gen_rnn_data(seq_skip)
    raw_text = '\n'.join(lines)

    # Creating an encoding for each encountered character
    chars = sorted(list(set(raw_text)))
    mapping = dict((c, i) for i, c in enumerate(chars))

    seqs = list()
    for line in lines:
        # Replace string with encoding of each character
        encoded_seq = [mapping[char] for char in line]
        # Replace list of sequences with list of encoded sequences
        seqs.append(encoded_seq)

    # Number of distinct characters
    vocab_size = len(mapping)

    # Splitting into input and output
    seqs = np.array(seqs)
    X, y = seqs[:, :-1], seqs[:, -1]

    # one-hot encoding
    seqs = [to_categorical(x, num_classes = vocab_size) for x in X]
    X = np.array(seqs)
    y = to_categorical(y, num_classes=vocab_size)

    # Model

    model = Sequential()
    model.add(LSTM(n_nodes, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(vocab_size, activation='softmax'))
    print(model.summary())

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=n_epochs, verbose=1)

    model.save(model_filename)

In [83]:
train_model(10, 200, 30, 'test_model_2.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 200)               217600    
_________________________________________________________________
dense_5 (Dense)              (None, 71)                14271     
Total params: 231,871
Trainable params: 231,871
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
9538/9538 [==============================] - 23s 2ms/step - loss: 3.0537 - acc: 0.1980
Epoch 2/30
9538/9538 [==============================] - 18s 2ms/step - loss: 2.7846 - acc: 0.2534
Epoch 3/30
9538/9538 [==============================] - 18s 2ms/step - loss: 2.5005 - acc: 0.3157
Epoch 4/30
9538/9538 [==============================] - 18s 2ms/step - loss: 2.4603 - acc: 0.3164
Epoch 5/30
9538/9538 [==============================] - 19s 2ms/step - loss: 2.3204 - acc: 0.3476
Epoch 6/30
9538/9538 [===============

In [77]:
def generate_text(model_filename, seed_text, n_chars):
    model = load_model(model_filename)
    for _ in range(n_chars):
        temp_seed = seed_text[-40:]
        encoded = [mapping[char] for char in temp_seed]
        encoded = array(to_categorical(encoded, num_classes=len(mapping)))
        encoded = np.reshape(encoded, (1, encoded.shape[0], encoded.shape[1]))

        yhat = model.predict_classes(encoded, verbose=0)

        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        seed_text += char
    return seed_text

In [87]:
generate_text('test_model_2.h5', 'From fairest creatures we desire increase', 45)

'From fairest creatures we desire increasend of thee mare the prome thee mane my leave '

{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'Y': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70}
